In [1]:
import tensorflow as tf
import numpy as np
#physical_devices = tf.config.experimental.list_physical_devices('GPU')

config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction=0.7
#gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=x)
#tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
session = tf.compat.v1.InteractiveSession(config=config)

In [2]:
Blinks = np.load('./npy/Blinks_F3.npy')
Labels = np.load('./npy/Labels_F3.npy')
BlinksTest = np.load('./npy/BlinksTest_F3.npy')
LabelsTest = np.load('./npy/LabelsTest_F3.npy')
    #deciding the indices of each video based on the fold
    #####################Normalizing the input#############Second phase
BlinksTest[:,:,0]=(BlinksTest[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
Blinks[:,:,0]=(Blinks[:,:,0]-np.mean(Blinks[:,:,0]))/np.std(Blinks[:,:,0])
    #####
    #####
BlinksTest[:,:,1]=(BlinksTest[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
Blinks[:,:,1]=(Blinks[:,:,1]-np.mean(Blinks[:,:,1]))/np.std(Blinks[:,:,1])
    #####
BlinksTest[:,:,2]=(BlinksTest[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
Blinks[:,:,2]=(Blinks[:,:,2]-np.mean(Blinks[:,:,2]))/np.std(Blinks[:,:,2])
    #####
BlinksTest[:,:,3]=(BlinksTest[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
Blinks[:,:,3]=(Blinks[:,:,3]-np.mean(Blinks[:,:,3]))/np.std(Blinks[:,:,3])
    ####
    ####JUST TO DOUBLE CHECK
    ####
    # print(np.mean(Blinks[:,:,0]),np.mean(Blinks[:,:,1]),np.mean(Blinks[:,:,2]),np.mean(Blinks[:,:,3]))
    # print(np.std(Blinks[:,:,0]),np.std(Blinks[:,:,1]),np.std(Blinks[:,:,2]),np.std(Blinks[:,:,3]))
    # print(np.mean(BlinksTest[:,:,0]),np.mean(BlinksTest[:,:,1]),np.mean(BlinksTest[:,:,2]),np.mean(BlinksTest[:,:,3]))
    # print(np.std(BlinksTest[:,:,0]),np.std(BlinksTest[:,:,1]),np.std(BlinksTest[:,:,2]),np.std(BlinksTest[:,:,3]))

In [4]:
def create_model(feature_size, post_fc2_size_per_layer=16, step_size=30):
    #Feature Transformation Layers
    def clean_split_and_squeeze(x, step_size):
        trash,output_RNN=tf.split(x,[step_size-1,1],axis=0,name='layers')  #size of output_RNN[lay] is (step,batch,hsize),
        return tf.squeeze(output_RNN,axis=0)

    input = tf.keras.Input(shape=(None, None, feature_size))
    #   F = ReLU(BW + b)
    F = tf.keras.layers.Dense(activation='relu', units=32)(tf.reshape(input, [-1, feature_size]))
    F = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(tf.reshape(F, [-1,step_size,32]))

    #SCATOLA NERA - LSTM (prima o poi la faremo)
    #lstm = tf.keras.layers.LSTM(4, return_sequences=True, return_state=True)
    #rnn_cells = [tf.keras.layers.LSTMCell(4) for _ in range(29)]
    #stacked_lstm = tf.keras.layers.StackedRNNCells(rnn_cells)
    #lstm_layer = tf.keras.layers.RNN(stacked_lstm, return_sequences=True, return_state=True)
    #output_RNN_set = lstm_layer(F)
    '''
    # Fully Connected Layer 1
    f1, f2, f3, f4 = clean_split_and_squeeze(output_RNN_set[0], step_size),\
                     clean_split_and_squeeze(output_RNN_set[1], step_size),\
                     clean_split_and_squeeze(output_RNN_set[2], step_size),\
                     clean_split_and_squeeze(output_RNN_set[3], step_size)
    f1post = tf.keras.layers.Dense(units=16)(f1)
    f2post = tf.keras.layers.Dense(units=16)(f2)
    f3post = tf.keras.layers.Dense(units=16)(f3)
    f4post = tf.keras.layers.Dense(units=16)(f4)
    post = tf.concat([f1post,f2post,f3post,f4post], axis=1)
    FC1 = tf.keras.layers.Dense(activation='relu', units=16)(post)
    FC1 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC1)

    #Fully Connected Layer 2

    FC2 = tf.keras.layers.Dense(activation='relu', units=16)(FC1)
    FC2 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC2)

    #Fully Connected Layer 3

    FC3 = tf.keras.layers.Dense(activation='relu', units=16)(FC2)
    FC3 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC3)

    #Fully Connected Layer 4

    FC4 = tf.keras.layers.Dense(activation='relu', units=8)(FC3)
    FC4 = tf.keras.layers.Dropout(rate=tf.Variable(1.0))(FC4)

    #Output Layer

    output = tf.keras.layers.Dense(units=1)(FC4)
    '''
    model = tf.keras.models.Model(inputs=[input], outputs=[F])
    return model

In [6]:
model = create_model(feature_size=4)
model.compile()

In [26]:
def test_model():
    input = tf.keras.Input(shape=(30, 4))
    x = tf.keras.layers.Masking(mask_value=0.0)(input)
    x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    y = tf.keras.layers.Dense(1, activation='tanh')(x)
    model = tf.keras.models.Model(inputs=[input], outputs=[y])
    return model
model2 = test_model()
model2.summary()
loss=tf.keras.losses.binary_crossentropy
optimizer=tf.keras.optimizers.Adam(learning_rate=0.00053)
model2.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Model: "functional_23"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_12 (InputLayer)        [(None, 30, 4)]           0         
_________________________________________________________________
masking_11 (Masking)         (None, 30, 4)             0         
_________________________________________________________________
lstm_11 (LSTM)               (None, 32)                4736      
_________________________________________________________________
dense_40 (Dense)             (None, 16)                528       
_________________________________________________________________
dropout_15 (Dropout)         (None, 16)                0         
_________________________________________________________________
dense_41 (Dense)             (None, 1)                 17        
Total params: 5,281
Trainable params: 5,281
Non-trainable params: 0
___________________________________________________

In [3]:
del model2

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 30, 4)]           0         
_________________________________________________________________
dense (Dense)                (None, 30, 64)            320       
_________________________________________________________________
masking (Masking)            (None, 30, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 32)                12416     
_________________________________________________________________
dense_1 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)               

In [ ]:
def resa_model():
    input = tf.keras.Input(shape=(30, 4))
    x = tf.keras.layers.Dense(64, activation='relu')(input)
    x = tf.keras.layers.Masking(mask_value=0.0)(x)
    x = tf.keras.layers.LSTM(32, return_sequences=False,dropout=0.1, recurrent_dropout=0.1)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    y = tf.keras.layers.Dense(1, activation='tanh')(x)
    model = tf.keras.models.Model(inputs=[input], outputs=[y])
    return model
with tf.device('/GPU:0'):
    model3 = resa_model()
    model3.summary()
    loss=tf.keras.losses.binary_crossentropy
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000053)
    model3.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [11]:
tf.executing_eagerly()

True

In [7]:
batch_size = 64
def resa_gpu():
    input = tf.keras.Input(shape=(30, 4))
    x = tf.keras.layers.Dense(64, activation='relu')(input)
    #x = tf.keras.layers.Masking(mask_value=0.0)(x)
    x = tf.keras.layers.LSTM(32,
                             activation="tanh",
                             recurrent_activation="sigmoid",
                             unroll=False,
                             use_bias=True,
                             return_sequences=False,
                             dropout=0.1,
                             recurrent_dropout=0.1,)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    x = tf.keras.layers.Dense(16, activation='relu')(x)
    x = tf.keras.layers.Dense(8, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    y = tf.keras.layers.Dense(1, activation='tanh')(x)
    model = tf.keras.models.Model(inputs=[input], outputs=[y])
    return model
with tf.device('/GPU:0'):
    model3 = resa_gpu()
    model3.summary()
    loss=tf.keras.losses.binary_crossentropy
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.000053)
    model3.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 30, 4)]           0         
_________________________________________________________________
dense_3 (Dense)              (None, 30, 64)            320       
_________________________________________________________________
lstm_3 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_4 (Dense)              (None, 16)                528       
_________________________________________________________________
dropout (Dropout)            (None, 16)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 16)                272       
_________________________________________________________________
dense_6 (Dense)              (None, 8)                

In [8]:
Labels2 = Labels
LabelsTest[LabelsTest==10] = 1
Labels2[Labels2==10] = 1

In [13]:
del model3

In [27]:
with tf.device('/GPU:0'):
    h2 = model2.fit(x= Blinks, y = Labels2, validation_data=(BlinksTest, LabelsTest), epochs=100, batch_size=64)

Epoch 1/100
121/121 [==============================] - 10s 82ms/step - loss: 1.6884 - accuracy: 0.5022 - val_loss: 0.6867 - val_accuracy: 0.6673

In [9]:
with tf.device('/GPU:0'):
    h3 = model3.fit(x= Blinks, y = Labels2, validation_data=(BlinksTest, LabelsTest), epochs=100, batch_size=batch_size)

Epoch 1/100
121/121 [==============================] - 12s 96ms/step - loss: 5.0574 - accuracy: 0.4588 - val_loss: 5.7458 - val_accuracy: 0.2994

In [10]:
model3.evaluate(x= Blinks, y = Labels2, batch_size=64)

121/121 [==============================] - 1s 11ms/step - loss: 0.5136 - accuracy: 0.7262

[0.5135590434074402, 0.726215124130249]

In [11]:
model3.evaluate(x = BlinksTest, y = LabelsTest)

71/71 [==============================] - 1s 12ms/step - loss: 0.4834 - accuracy: 0.7681

[0.48341014981269836, 0.7681030631065369]